# **1. 서울 열린데이터 광장**
[서울 열린데이터 광장(Seoul Open Data Plaza) ](https://data.seoul.go.kr/)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

# **2. 서울시 공공자전거 실시간 대여정보**

[링크](https://data.seoul.go.kr/dataList/OA-15493/A/1/datasetView.do)

In [1]:
import requests
import pandas as pd
import folium

In [2]:
base_url = 'http://openapi.seoul.go.kr:8088/6c4a67496d6d696e36367652764d59/json/bikeList/1/2/'
response = requests.get(base_url)
response

<Response [200]>

In [3]:
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 2,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '9',
    'shared': '60',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '22',
    'shared': '157',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'}]}}

In [4]:
json_data.get('rentBikeStatus', {})

{'list_total_count': 2,
 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
 'row': [{'rackTotCnt': '15',
   'stationName': '102. 망원역 1번출구 앞',
   'parkingBikeTotCnt': '9',
   'shared': '60',
   'stationLatitude': '37.55564880',
   'stationLongitude': '126.91062927',
   'stationId': 'ST-4'},
  {'rackTotCnt': '14',
   'stationName': '103. 망원역 2번출구 앞',
   'parkingBikeTotCnt': '22',
   'shared': '157',
   'stationLatitude': '37.55495071',
   'stationLongitude': '126.91083527',
   'stationId': 'ST-5'}]}

In [5]:
json_data.get('rentBikeStatus', {}).get('RESULT', {})

{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'}

In [6]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", {})

'INFO-000'

In [7]:
def fetch_bike_data():
    base_url = "http://openapi.seoul.go.kr:8088/6c4a67496d6d696e36367652764d59/json/bikeList/"
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f"{base_url}{start}/{end}/"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break

        json_data = response.json()

        try:
            rent_bike_status = json_data["rentBikeStatus"]
            result_code = rent_bike_status["RESULT"]["CODE"]
        except KeyError:
            print("JSON 오류")
            break

        if result_code == "INFO-200":
            print("데이터 없음")
            break
        elif result_code == "INFO-000":
            print(f"시작: {start} 끝: {end}.")
            try:
                bike_data = rent_bike_status["row"]
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print("데이터를 찾을 수 없음")
        else:
            print(f"result code: {result_code}")
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [8]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


In [9]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,9,60,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,22,157,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,1,8,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,0,0,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,1,8,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2722,10,6177. 마곡롯데캐슬르웨스트,1,10,37.56356049,126.82462311,ST-3324
2723,10,6178. 마스터밸류에이스 지식산업센터,2,20,37.55875015,126.85991669,ST-3337
2724,9,6180.양천향교역 3번출구,19,211,37.56978226,126.84175110,ST-3365
2725,10,6182.오스템임플란트비전타워,25,250,37.57143402,126.82884979,ST-3370


In [10]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727 entries, 0 to 2726
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2727 non-null   object
 1   stationName        2727 non-null   object
 2   parkingBikeTotCnt  2727 non-null   object
 3   shared             2727 non-null   object
 4   stationLatitude    2727 non-null   object
 5   stationLongitude   2727 non-null   object
 6   stationId          2727 non-null   object
dtypes: object(7)
memory usage: 149.3+ KB


```
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름

```


In [11]:
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [12]:
bike_data_df.shape

(2727, 7)

In [13]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [14]:
 bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map

Output hidden; open in https://colab.research.google.com to view.